# 1. Nettoyage des données

In [18]:
import pandas as pd

df = pd.read_csv('CRSP-data.csv')
#print(df)
df.head()

,PERMNO,date,PRIMEXCH,RET,semester
0,10001,19851231,NaN,NaN,42
1,10001,19860131,Q,C,43
2,10001,19860228,Q,0.020408,43
3,10001,19860331,Q,0.025200,43
4,10001,19860430,Q,0.009901,43


In [19]:
# élimination des données manquantes

df.dropna(axis=0, inplace =True)
#print(df)
df.head()

,PERMNO,date,PRIMEXCH,RET,semester
1,10001,19860131,Q,C,43
2,10001,19860228,Q,0.020408,43
3,10001,19860331,Q,0.025200,43
4,10001,19860430,Q,0.009901,43
5,10001,19860530,Q,-0.009804,43


In [20]:
# élimination des actions cotées en dehors du NYSE et de l’AMEX

lieux_echanges = ['N','A']
    
df_cleaned = df[df['PRIMEXCH'].isin(lieux_echanges)]
print(df_cleaned.head())

# conversion de la colonne 'date' en format datetie YYYYMMDD
print(df_cleaned.dtypes)
df_cleaned.loc[:,'date']=pd.to_datetime(df_cleaned.loc[:,'date'],format='%Y%m%d')
print(df_cleaned.head())

     PERMNO      date PRIMEXCH        RET  semester
147   10006  19650129        N   0.053459         1
148   10006  19650226        N  -0.025970         1
149   10006  19650331        N   0.040248         1
150   10006  19650430        N   0.084821         1
151   10006  19650528        N  -0.043621         1
PERMNO       int64
date         int64
PRIMEXCH    object
RET         object
semester     int64
dtype: object
     PERMNO       date PRIMEXCH        RET  semester
147   10006 1965-01-29        N   0.053459         1
148   10006 1965-02-26        N  -0.025970         1
149   10006 1965-03-31        N   0.040248         1
150   10006 1965-04-30        N   0.084821         1
151   10006 1965-05-28        N  -0.043621         1


C:\Users\eloim\AppData\Local\Temp\ipykernel_26632\2948778794.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.loc[:,'date']=pd.to_datetime(df_cleaned.loc[:,'date'],format='%Y%m%d')


# 2. Constitution des portefeuilles Loosers et Winners

In [21]:
df_cleaned = df_cleaned.copy()

# on renomme la colonne RET en TRC (Taux de Rentabilité Cumulés)
df_cleaned.rename(columns={'RET':'TRC'}, inplace =True)
print(df_cleaned.columns)

# Conversion du type de la colonne TRC en float
df_cleaned['TRC'] = pd.to_numeric(df_cleaned['TRC'], errors='coerce')
print(df_cleaned['TRC'].dtype)

# Suppression des lignes où la conversion a échoué
df_cleaned.dropna(subset=['TRC'], inplace=True)

semester_period = 1

# Filtrage des données sur une période de 12 mois
df_period = df_cleaned[df_cleaned['semester']<=semester_period+1]
print(df_period.head())


# calcul du taux de rentabilité cumulé de chaque titre sur la période de 12 mois
df_group = df_period.groupby('PERMNO')

result = df_group['TRC'].sum()
print(result.head())
#result devient une serie


Index(['PERMNO', 'date', 'PRIMEXCH', 'TRC', 'semester'], dtype='object')
float64
     PERMNO       date PRIMEXCH       TRC  semester
147   10006 1965-01-29        N  0.053459         1
148   10006 1965-02-26        N -0.025970         1
149   10006 1965-03-31        N  0.040248         1
150   10006 1965-04-30        N  0.084821         1
151   10006 1965-05-28        N -0.043621         1
PERMNO
10006    0.237466
10030    0.111438
10057    0.391155
10065    0.113284
10102    0.383250
Name: TRC, dtype: float64


In [22]:
pd.set_option('display.max_rows', None)

# Afficher toutes les colonnes
pd.set_option('display.max_columns', None)

import numpy as np

# Tri décroissant de la serie par TRC
sorted_result = result.sort_values(ascending=False)

# Conversion de la série en DataFrame
result_df = sorted_result.to_frame().reset_index()

# Tri du DataFrame par la colonne 'TRC'
result_df = result_df.sort_values(by='TRC', ascending=False).reset_index(drop=True)


# Ajout de la nouvelle colonne 'Decile'
result_df['Decile'] = 1

# Création d'une liste de sous-DataFrame divisée de manière égale en quantité
dix_decile = np.array_split(result_df,10)
    
# Attribution d'une valeur entière comprise entre 0 et 9 pour chaque PERMNO
p=len(dix_decile)-1
for i in range(0,len(dix_decile),1):
    dix_decile[p]['Decile'] = i
    p-=1

# Création des déciles avec pd.qcut
# result_df['Decile'] = pd.qcut(result_df['TRC'], q=10, labels=False)

# Sélection des titres appartenant aux déciles 0 (Loosers) et 9 (Winners)
df_selected = result_df[result_df['Decile'].isin([0, 9])]

# Concaténer les sous-DataFrames ensemble
result_df = pd.concat(dix_decile, ignore_index=True)

# Sélection des titres appartenant aux déciles 0 (Loosers) et 9 (Winners)
win_loose = [0,9]

df_selected = result_df[result_df['Decile'].isin(win_loose)]

print(df_selected.head())


   PERMNO       TRC  Decile
0   31579  3.069201       9
1   38156  2.781328       9
2   34569  2.739458       9
3   41072  1.966338       9
4   28169  1.952610       9


# 3.Calcul de la rentabilité sur les 6 mois suivants

In [23]:
import numpy as np
import pandas as pd

# Filtrer les données sur la période de six mois
df_period = df_cleaned[df_cleaned['semester'] == semester_period + 2]

# Calcul du taux de rentabilité cumulé sur la période de six mois
df_group = df_period.groupby('PERMNO')['TRC'].sum()
result_df = df_group.to_frame().reset_index()

# Trier par TRC de manière décroissante
sorted_result = result_df.sort_values(by='TRC', ascending=False).reset_index(drop=True)

# Attribution des déciles sur les données de la période de six mois
dix_decile = np.array_split(sorted_result, 10)

# Ajouter la colonne 'Decile'
for i in range(0, len(dix_decile)):
    dix_decile[i]['Decile'] = i

# Concaténer les sous-DataFrames
sorted_result = pd.concat(dix_decile, ignore_index=True)

# Sélection des titres appartenant aux déciles 0 et 9
df_selected = sorted_result[sorted_result['Decile'].isin([0, 9])]

# Merge avec les données de la période de six mois
df_final = pd.merge(df_period, df_selected[['PERMNO', 'Decile']], on='PERMNO', how='inner')
df_final.rename(columns={'TRC': 'TRC 6m after period'}, inplace=True)

# Vérifier les résultats
print(df_final.head())
print("Maximum TRC pour le décile 9 :", df_final[df_final['Decile'] == 9]['TRC 6m after period'].max())


   PERMNO       date PRIMEXCH  TRC 6m after period  semester  Decile
0   10620 1966-01-31        N             0.084746         3       9
1   10620 1966-02-28        N            -0.043750         3       9
2   10620 1966-03-31        N            -0.060440         3       9
3   10620 1966-04-29        N            -0.099415         3       9
4   10620 1966-05-31        N            -0.041558         3       9
Maximum TRC pour le décile 9 : 0.347826


In [24]:
mean_TRC_winner=0.0
count_winner=0
mean_TRC_looser =0.0
count_looser=0
for i in range (0,len(df_final),1):
    if df_final.iloc[i]['Decile']==9:
        mean_TRC_winner+=df_final.iloc[i]['TRC 6m after period']
        count_winner+=1
    elif df_final.iloc[i]['Decile'] == 0:
        mean_TRC_looser+=df_final.iloc[i]['TRC 6m after period']
        count_looser+=1
        
mean_TRC_winner = mean_TRC_winner/count_winner
mean_TRC_looser = mean_TRC_looser/count_looser
print("Rentabilité moyenne des Winners :", mean_TRC_winner)
print("Rentabilité moyenne des Loosers :", mean_TRC_looser)


retMom = []
retMom.append(mean_TRC_winner-mean_TRC_looser)
print("Différentiel de rentabilité (Momentum) :", retMom)

Rentabilité moyenne des Winners : -0.04904246759259257
Rentabilité moyenne des Loosers : 0.08532638650865988
Différentiel de rentabilité (Momentum) : [-0.13436885410125246]


# 4.Itération et test de significativité 

In [25]:
max_value = df_cleaned['semester'].max()

for i in range (3, max_value, 1):
    # Filtrage des données sur la période de six mois suivant celle de 12 mois
    df_period = df_cleaned[df_cleaned['semester']==semester_period+i]
    #print(df_period)
    
    # calcul du taux de rentabilité cumulé de chaque titre sur la période de 6 mois
    df_group = df_period.groupby('PERMNO')
    
    result = df_group['TRC'].sum()
    #print(result)
    
    # Conversion de la série en DataFrame
    result_df = result.to_frame().reset_index()
    
    df_final = pd.merge(result_df,df_selected[['PERMNO','Decile']],on='PERMNO',how='inner' )
    df_final.rename(columns={'TRC':'TRC 6m after period'}, inplace =True)
    #print(df_final)
    
    mean_TRC_winner=0.0
    count_winner=0
    mean_TRC_looser =0.0
    count_looser=0
    for j in range (0,len(df_final),1):
        if df_final.iloc[j]['Decile']==0:
            mean_TRC_winner+=df_final.iloc[j]['TRC 6m after period']
            count_winner+=1
        else : 
            mean_TRC_looser+=df_final.iloc[j]['TRC 6m after period']
            count_looser+=1
        
    mean_TRC_winner = mean_TRC_winner/count_winner
    mean_TRC_looser = mean_TRC_looser/count_looser
    #print(mean_TRC_winner)
    #print(mean_TRC_looser)
    
    retMom.append(mean_TRC_winner-mean_TRC_looser)

print(retMom)

[-0.13436885410125246, -0.03734884428141694, 0.01678635265151518, 0.08538864238095242, -0.10479148858008719, -0.03879860544354832, -0.05819120262353544, -0.06348394027210887, -0.10925711903637225, -0.033761937103174544, -0.021726246427469925, -0.05991269072494669, 0.07188056313754966, -0.021872044893255043, 0.06813599515503876, 0.06882456818045114, 0.10761848764186634, -0.017235295986622212, -0.017934499032079887, -0.046186386614173236, 0.07234935862631636, -0.002336471027893375, 0.04208627030812333, 0.01896250708910885, 0.03968413917525773, 0.034403497916666685, 0.014592568717809506, 0.12091140961098408, 0.023802890957176304, 0.054937034224599, -0.017252280466148753, -0.028515566035570844, -0.015435611538461576, 0.0960328126085877, -0.023655903432076242, -0.027186273954116057, -0.07251784649122806, -0.037285960454545485, -0.023533034409654305, -0.05137572424910905, -0.1442051302513301, 0.05042574965034965, 0.04034502430555542, -0.07916186031746036, -0.012945371641791076, -0.1162092461